In [1]:
import tensorflow as tf
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

: /physical_device:CPU:0
: /physical_device:GPU:0


In [2]:
# Import all necessary libraries
import os
import logging
import warnings
import numpy as np
import matplotlib.pyplot as plt

# Suppress warnings and logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Define constants
IMAGE_SIZE = (224, 224)  # Directly using ResNet50 input size
BATCH_SIZE = 64
TRAINING_DATA_PATH = "Final_Arabic_Alpha_dataset/train"
TEST_DATA_PATH = "Final_Arabic_Alpha_dataset/test"

# Load datasets
print("Loading datasets...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAINING_DATA_PATH,
    validation_split=0.1,
    subset="training",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    label_mode='int'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAINING_DATA_PATH,
    validation_split=0.1,
    subset="validation",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    label_mode='int'
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DATA_PATH,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    label_mode=None,
    shuffle=False
)

# Get class names
class_names = train_ds.class_names
print(f"Class names: {class_names}")


AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

Loading datasets...
Found 42559 files belonging to 65 classes.
Using 38304 files for training.
Found 42559 files belonging to 65 classes.
Using 4255 files for validation.
Found 10640 files belonging to 1 classes.
Class names: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '7', '8', '9']


In [3]:
# Data augmentation and preprocessing
print("Creating preprocessing pipeline...")
# data_augmentation = tf.keras.Sequential([
#     tf.keras.layers.RandomRotation(0.2, seed=SEED),
#     tf.keras.layers.RandomZoom(0.1, seed=SEED),
# ])

resnet_preprocess = tf.keras.applications.resnet50.preprocess_input

# Build model
print("Building model...")
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMAGE_SIZE, 3)
)

inputs = tf.keras.Input(shape=(*IMAGE_SIZE, 3))
# x = data_augmentation(inputs)
x = inputs
x = resnet_preprocess(x)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

# Freeze base model initially
base_model.trainable = False

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

model.summary()

Creating preprocessing pipeline...
Building model...
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (

In [4]:
# Callbacks
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=2,
        min_lr=1e-7
    )
]

EPOCHS = 5

# Train model
print("Training model...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

Training model...
Epoch 1/5
599/599 [==============================] - 162s 206ms/step - loss: 1.3761 - accuracy: 0.6104 - val_loss: 0.8457 - val_accuracy: 0.7358 - lr: 0.0010
Epoch 2/5
599/599 [==============================] - 116s 192ms/step - loss: 0.6356 - accuracy: 0.8026 - val_loss: 0.6940 - val_accuracy: 0.7793 - lr: 0.0010
Epoch 3/5
599/599 [==============================] - 115s 192ms/step - loss: 0.4772 - accuracy: 0.8486 - val_loss: 0.5988 - val_accuracy: 0.8094 - lr: 0.0010
Epoch 4/5
599/599 [==============================] - 116s 193ms/step - loss: 0.3865 - accuracy: 0.8771 - val_loss: 0.5453 - val_accuracy: 0.8310 - lr: 0.0010
Epoch 5/5
599/599 [==============================] - 116s 194ms/step - loss: 0.3331 - accuracy: 0.8928 - val_loss: 0.5200 - val_accuracy: 0.8364 - lr: 0.0010


In [5]:
# # Unfreeze some layers of the base model
# fine_tune_at = 150  # Unfreeze from this layer onwards

# base_model.trainable = True
# for layer in base_model.layers[:fine_tune_at]:
#     layer.trainable = False

model.optimizer.learning_rate.assign(1e-5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-05>

In [6]:
# Train the model again with fine-tuning
fine_tune_epochs = 10  # Fine-tune for 10 more epochs
total_epochs = EPOCHS + fine_tune_epochs  # Add to previous epochs

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],  # Resume from where we left off
    callbacks=callbacks
)

Epoch 5/15


599/599 [==============================] - 116s 193ms/step - loss: 0.2244 - accuracy: 0.9332 - val_loss: 0.4321 - val_accuracy: 0.8639 - lr: 1.0000e-05
Epoch 6/15
599/599 [==============================] - 138s 231ms/step - loss: 0.2023 - accuracy: 0.9420 - val_loss: 0.4226 - val_accuracy: 0.8656 - lr: 1.0000e-05
Epoch 7/15
599/599 [==============================] - 159s 265ms/step - loss: 0.1967 - accuracy: 0.9439 - val_loss: 0.4178 - val_accuracy: 0.8672 - lr: 1.0000e-05
Epoch 8/15
599/599 [==============================] - 156s 259ms/step - loss: 0.1929 - accuracy: 0.9456 - val_loss: 0.4145 - val_accuracy: 0.8682 - lr: 1.0000e-05
Epoch 9/15
599/599 [==============================] - 147s 244ms/step - loss: 0.1899 - accuracy: 0.9466 - val_loss: 0.4119 - val_accuracy: 0.8684 - lr: 1.0000e-05
Epoch 10/15
599/599 [==============================] - 138s 230ms/step - loss: 0.1874 - accuracy: 0.9476 - val_loss: 0.4099 - val_accuracy: 0.8686 - lr: 1.0000e-05
Epoch 11/15
599/599 [===========

In [3]:
# Load the model
loaded_model = tf.keras.models.load_model("arabic_letter_classifier_95_87")
loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544

In [ ]:
loss, accuracy = loaded_model.evaluate(train_ds)
val_loss, val_accuracy = loaded_model.evaluate(val_ds)

print(f"Training loss: {loss}, Training Accuracy: {accuracy}")
print(f"Val loss: {val_loss}, Val Accuracy: {val_accuracy}")

67/67 [==============================] - 13s 186ms/step - loss: 0.4037 - accuracy: 0.8714
Training loss: 0.1766589730978012, Training Accuracy: 0.9521459937095642
Val loss: 0.40367570519447327, Val Accuracy: 0.87144535779953


In [5]:
base_model = loaded_model.get_layer("resnet50")

# Unfreeze some layers of the base model
fine_tune_at = 100  # Unfreeze from this layer onwards

base_model.trainable = True
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

loaded_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               524544

In [6]:
# Callbacks
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=2,
        min_lr=1e-7
    )
]


# Train the model again with fine-tuning
fine_tune_epochs = 5  # Fine-tune for 10 more epochs
# total_epochs_final = EPOCHS + total_epochs  # Add to previous epochs

history_fine_final = loaded_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs,
    callbacks=callbacks
)

Epoch 1/5
599/599 [==============================] - 193s 315ms/step - loss: 1.9411 - accuracy: 0.5785 - val_loss: 1.1386 - val_accuracy: 0.7001 - lr: 1.0000e-06
Epoch 2/5
599/599 [==============================] - 189s 315ms/step - loss: 0.7661 - accuracy: 0.7711 - val_loss: 0.8267 - val_accuracy: 0.7702 - lr: 1.0000e-06
Epoch 3/5
599/599 [==============================] - 189s 315ms/step - loss: 0.5357 - accuracy: 0.8318 - val_loss: 0.6937 - val_accuracy: 0.8005 - lr: 1.0000e-06
Epoch 4/5
599/599 [==============================] - 195s 325ms/step - loss: 0.4190 - accuracy: 0.8661 - val_loss: 0.6149 - val_accuracy: 0.8183 - lr: 1.0000e-06
Epoch 5/5
599/599 [==============================] - 193s 322ms/step - loss: 0.3364 - accuracy: 0.8897 - val_loss: 0.5614 - val_accuracy: 0.8310 - lr: 1.0000e-06


In [7]:
history_fine_final = loaded_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs+5,
    initial_epoch=5,
    callbacks=callbacks
)

Epoch 6/10
599/599 [==============================] - 192s 321ms/step - loss: 0.2813 - accuracy: 0.9073 - val_loss: 0.5235 - val_accuracy: 0.8428 - lr: 1.0000e-06
Epoch 7/10
599/599 [==============================] - 191s 318ms/step - loss: 0.2363 - accuracy: 0.9244 - val_loss: 0.4941 - val_accuracy: 0.8505 - lr: 1.0000e-06
Epoch 8/10
599/599 [==============================] - 192s 320ms/step - loss: 0.1995 - accuracy: 0.9365 - val_loss: 0.4723 - val_accuracy: 0.8562 - lr: 1.0000e-06
Epoch 9/10
599/599 [==============================] - 219s 366ms/step - loss: 0.1725 - accuracy: 0.9468 - val_loss: 0.4543 - val_accuracy: 0.8623 - lr: 1.0000e-06
Epoch 10/10
599/599 [==============================] - 192s 320ms/step - loss: 0.1482 - accuracy: 0.9550 - val_loss: 0.4395 - val_accuracy: 0.8660 - lr: 1.0000e-06


In [9]:
for layer in loaded_model.layers:
    if layer.name == "dense":
        layer.kernel_regularizer = tf.keras.regularizers.l2(1e-3)

history_fine_final = loaded_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs+5+5,
    initial_epoch=10,
    callbacks=callbacks
)

Epoch 11/15
599/599 [==============================] - 187s 311ms/step - loss: 0.1276 - accuracy: 0.9618 - val_loss: 0.4286 - val_accuracy: 0.8731 - lr: 1.0000e-06
Epoch 12/15
599/599 [==============================] - 189s 316ms/step - loss: 0.1096 - accuracy: 0.9697 - val_loss: 0.4183 - val_accuracy: 0.8757 - lr: 1.0000e-06
Epoch 13/15
599/599 [==============================] - 192s 320ms/step - loss: 0.0946 - accuracy: 0.9736 - val_loss: 0.4109 - val_accuracy: 0.8808 - lr: 1.0000e-06
Epoch 14/15
599/599 [==============================] - 190s 318ms/step - loss: 0.0806 - accuracy: 0.9792 - val_loss: 0.4045 - val_accuracy: 0.8808 - lr: 1.0000e-06
Epoch 15/15
599/599 [==============================] - 189s 316ms/step - loss: 0.0702 - accuracy: 0.9834 - val_loss: 0.3975 - val_accuracy: 0.8834 - lr: 1.0000e-06


In [10]:
for layer in loaded_model.layers:
    if layer.name == "dense":
        layer.kernel_regularizer = tf.keras.regularizers.l2(1e-1)

history_fine_final = loaded_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs+5+5+5,
    initial_epoch=15,
    callbacks=callbacks
)

Epoch 16/20
599/599 [==============================] - 188s 312ms/step - loss: 0.0599 - accuracy: 0.9862 - val_loss: 0.3934 - val_accuracy: 0.8855 - lr: 1.0000e-06
Epoch 17/20
599/599 [==============================] - 186s 310ms/step - loss: 0.0514 - accuracy: 0.9900 - val_loss: 0.3879 - val_accuracy: 0.8886 - lr: 1.0000e-06
Epoch 18/20
599/599 [==============================] - 186s 311ms/step - loss: 0.0446 - accuracy: 0.9920 - val_loss: 0.3848 - val_accuracy: 0.8884 - lr: 1.0000e-06
Epoch 19/20
599/599 [==============================] - 301s 502ms/step - loss: 0.0382 - accuracy: 0.9942 - val_loss: 0.3803 - val_accuracy: 0.8902 - lr: 1.0000e-06
Epoch 20/20
599/599 [==============================] - 187s 312ms/step - loss: 0.0332 - accuracy: 0.9954 - val_loss: 0.3792 - val_accuracy: 0.8910 - lr: 1.0000e-06


In [11]:
for layer in loaded_model.layers:
    if hasattr(layer, "kernel_regularizer"):
        layer.kernel_regularizer = tf.keras.regularizers.l2(1)

history_fine_final = loaded_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs+5+5+5+5,
    initial_epoch=20,
    callbacks=callbacks
)

Epoch 21/25
599/599 [==============================] - 187s 311ms/step - loss: 0.0290 - accuracy: 0.9963 - val_loss: 0.3780 - val_accuracy: 0.8921 - lr: 1.0000e-06
Epoch 22/25
599/599 [==============================] - 211s 351ms/step - loss: 0.0248 - accuracy: 0.9978 - val_loss: 0.3739 - val_accuracy: 0.8940 - lr: 1.0000e-06
Epoch 23/25
599/599 [==============================] - 187s 312ms/step - loss: 0.0213 - accuracy: 0.9986 - val_loss: 0.3723 - val_accuracy: 0.8961 - lr: 1.0000e-06
Epoch 24/25
599/599 [==============================] - 193s 323ms/step - loss: 0.0188 - accuracy: 0.9989 - val_loss: 0.3728 - val_accuracy: 0.8964 - lr: 1.0000e-06
Epoch 25/25
599/599 [==============================] - 187s 312ms/step - loss: 0.0162 - accuracy: 0.9995 - val_loss: 0.3729 - val_accuracy: 0.8973 - lr: 1.0000e-06


In [12]:
for layer in loaded_model.layers:
    if hasattr(layer, "kernel_regularizer"):
        layer.kernel_regularizer = tf.keras.regularizers.l2(2)

history_fine_final = loaded_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs+5+5+5+5+5,
    initial_epoch=25,
    callbacks=callbacks
)

Epoch 26/30
599/599 [==============================] - 196s 326ms/step - loss: 0.0138 - accuracy: 0.9996 - val_loss: 0.3729 - val_accuracy: 0.8987 - lr: 2.0000e-07
Epoch 27/30
599/599 [==============================] - 187s 312ms/step - loss: 0.0135 - accuracy: 0.9995 - val_loss: 0.3727 - val_accuracy: 0.8982 - lr: 2.0000e-07
Epoch 28/30
599/599 [==============================] - 186s 311ms/step - loss: 0.0125 - accuracy: 0.9997 - val_loss: 0.3724 - val_accuracy: 0.8999 - lr: 2.0000e-07
Epoch 29/30
599/599 [==============================] - 186s 311ms/step - loss: 0.0124 - accuracy: 0.9997 - val_loss: 0.3719 - val_accuracy: 0.8999 - lr: 2.0000e-07
Epoch 30/30
599/599 [==============================] - 186s 311ms/step - loss: 0.0121 - accuracy: 0.9998 - val_loss: 0.3716 - val_accuracy: 0.8999 - lr: 2.0000e-07


In [15]:
history_fine_final = loaded_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=fine_tune_epochs+5+5+5+5+5+5,
    initial_epoch=30,
    callbacks=callbacks
)

Epoch 31/35
599/599 [==============================] - 192s 319ms/step - loss: 0.0118 - accuracy: 0.9998 - val_loss: 0.3711 - val_accuracy: 0.8996 - lr: 2.0000e-07
Epoch 32/35
599/599 [==============================] - 188s 314ms/step - loss: 0.0110 - accuracy: 0.9998 - val_loss: 0.3710 - val_accuracy: 0.9004 - lr: 2.0000e-07
Epoch 33/35
599/599 [==============================] - 189s 316ms/step - loss: 0.0108 - accuracy: 0.9997 - val_loss: 0.3715 - val_accuracy: 0.9008 - lr: 2.0000e-07
Epoch 34/35
599/599 [==============================] - 189s 315ms/step - loss: 0.0104 - accuracy: 0.9997 - val_loss: 0.3709 - val_accuracy: 0.9008 - lr: 2.0000e-07
Epoch 35/35
599/599 [==============================] - 186s 311ms/step - loss: 0.0100 - accuracy: 0.9997 - val_loss: 0.3707 - val_accuracy: 0.9018 - lr: 2.0000e-07


In [16]:
loaded_model.save('arabic_letter_classifier_100_90')